In [138]:
import xlsxwriter
import pandas
import re

In [139]:
csv_file_name = 'results_total.csv'
data = pandas.read_csv(csv_file_name, sep = ';', encoding='latin-1')
data_dictionary = data.to_dict()
print(data_dictionary.keys())
print('Full size: {0}'.format(len(data_dictionary['Status'])))
# печать 0-ой записи, value[0] - 0-ая запись
for key, value in data_dictionary.items():
    print(key, value[0])

dict_keys(['Status', 'Task type', 'Topology name', 'Dataset', 'Framework', 'Inference Framework', 'Input blob sizes', 'Precision', 'Batch size', 'Mode', 'Parameters', 'Infrastructure', 'Average time of single pass (s)', 'Latency', 'FPS'])
Full size: 3528
Status Success
Task type classification
Topology name alexnet
Dataset ImageNET
Framework Caffe
Inference Framework OpenVINO DLDT
Input blob sizes 1x3x227x227
Precision FP32
Batch size 1
Mode Sync
Parameters Device: CPU, Iteration count: 1000
Infrastructure CPU: Intel(R) Xeon(R) Gold 6138 CPU @ 2.00GHz CPU family: x86_64, GPU: None, RAM size: 526980036 kB, OS family: Linux OS version: Linux-5.8.0-53-generic-x86_64-with-Ubuntu-18.04-bionic, Python version: 3.6.9
Average time of single pass (s) 0.004
Latency 0.004
FPS 269.304


In [140]:
excel_file_name = 'test.xlsx'
excel_sheet_name = 'Performance'
book = xlsxwriter.Workbook(excel_file_name)
sheet = book.add_worksheet(excel_sheet_name)

sheet_head = list(data_dictionary.keys())

cell_format = book.add_format({'align': 'center', 'valign': 'vcenter', 'border': 1, 'bold': True})
sheet.merge_range('A1:A5', sheet_head[1], cell_format) # Task type
sheet.merge_range('B1:B5', sheet_head[2], cell_format) # Topology name
sheet.merge_range('C1:C5', sheet_head[4], cell_format) # Framework
sheet.merge_range('D1:D5', sheet_head[6], cell_format) # Input blob sizes
sheet.merge_range('E1:E5', sheet_head[8], cell_format) # Batch size

In [141]:
# get unique infrastructure
infrastructure = list(set(list(data_dictionary['Infrastructure'].values())))
inference_frameworks = []

for machine in infrastructure:
    machine_inference_frameworks = []
    for key, value in data_dictionary['Inference Framework'].items():
        if data_dictionary['Infrastructure'][key] == machine and \
           (value in machine_inference_frameworks) == False:
            machine_inference_frameworks.append(value)
    inference_frameworks.append(machine_inference_frameworks)

devices = []
for idx in range(len(infrastructure)):
    machine = infrastructure[idx]
    machine_inference_frameworks = inference_frameworks[idx]
    #print(machine)
    #print(machine_inference_frameworks)
    machine_framework_devices = []
    for inference_framework in machine_inference_frameworks:
        #print(inference_framework)
        framework_devices = []
        for key, value in data_dictionary['Parameters'].items():
            pattern = re.compile(r'[.]*Device:[ ]*(?P<device_name>[\W\w]+)[,]+[.]*')
            matcher = re.match(pattern, value)
            device_name = matcher.group('device_name')
            if data_dictionary['Infrastructure'][key] == machine and \
               data_dictionary['Inference Framework'][key] == inference_framework and \
               (device_name in framework_devices) == False:
                framework_devices.append(device_name)
        machine_framework_devices.append(framework_devices)
    devices.append(machine_framework_devices)

print(infrastructure)
print(inference_frameworks)
print(devices)

['CPU: Intel(R) Core(TM) i3-8100 CPU @ 3.60GHz, CPU family: x86_64, GPU: Intel(R) Gen9 HD Graphics (iGPU), RAM size: 32757700 kB, OS family: Linux, OS version: Linux-5.4.0-73-generic-x86_64-with-Ubuntu-18.04-bionic, Python version: 3.6.9', 'CPU: Intel(R) Core(TM) i7-8700 CPU @ 3.20GHz, CPU family: x86_64, GPU: Intel(R) Gen9 HD Graphics (iGPU), RAM size: 65727428 kB, OS family: Linux, OS version: Linux-5.4.0-52-generic-x86_64-with-Ubuntu-18.04-bionic, Python version: 3.6.9', 'CPU: Intel(R) Xeon(R) Gold 6138 CPU @ 2.00GHz CPU family: x86_64, GPU: None, RAM size: 526980036 kB, OS family: Linux OS version: Linux-5.8.0-53-generic-x86_64-with-Ubuntu-18.04-bionic, Python version: 3.6.9']
[['OpenVINO DLDT'], ['OpenVINO DLDT'], ['OpenVINO DLDT']]
[[['CPU', 'GPU', 'MYRIAD']], [['CPU', 'GPU', 'MYRIAD']], [['CPU']]]


In [142]:
precisions = []
for idx in range(len(infrastructure)):
    machine = infrastructure[idx]
    machine_inference_frameworks = inference_frameworks[idx]
    machine_precisions = []
    #print(machine)
    #print(machine_inference_frameworks)
    for idx2 in range(len(machine_inference_frameworks)):
        inference_framework = machine_inference_frameworks[idx2]
        framework_devices = devices[idx][idx2]
        framework_precisions = []
        #print(inference_framework)
        #print(framework_devices)
        for device in framework_devices:
            #print(device)
            device_precisions = []
            for key, value in data_dictionary['Parameters'].items():
                pattern = re.compile(r'[.]*Device:[ ]*(?P<device_name>[\W\w]+)[,]+[.]*')
                matcher = re.match(pattern, value)
                device_name = matcher.group('device_name')
                #print(data_dictionary['Infrastructure'][key], data_dictionary['Inference Framework'][key], device_name, data_dictionary['Precision'][key])
                if data_dictionary['Infrastructure'][key] == machine and \
                   data_dictionary['Inference Framework'][key] == inference_framework and \
                   device_name == device and \
                   (data_dictionary['Precision'][key] in device_precisions) == False:
                    device_precisions.append(data_dictionary['Precision'][key])
            #print(device_precisions)
            framework_precisions.append(device_precisions)
        machine_precisions.append(framework_precisions)
    precisions.append(machine_precisions)           

print(precisions)   


[[[['FP32'], ['FP32', 'FP16'], ['FP16']]], [[['FP32'], ['FP32', 'FP16'], ['FP16']]], [[['FP32']]]]


In [143]:
execution_modes = []
for idx in range(len(infrastructure)):
    machine = infrastructure[idx]
    machine_inference_frameworks = inference_frameworks[idx]
    #print(machine)
    #print(machine_inference_frameworks)
    machine_modes = []
    for idx2 in range(len(machine_inference_frameworks)):
        inference_framework = machine_inference_frameworks[idx2]
        framework_devices = devices[idx][idx2]
        #print(inference_framework)
        #print(framework_devices)
        framework_modes = []
        for idx3 in range(len(framework_devices)):
            device = framework_devices[idx3]
            device_precisions = precisions[idx][idx2][idx3]
            #print(device)
            #print(device_precisions)
            framework_device_modes = []
            for precision in device_precisions:
                #print(precision)
                device_precision_modes = []
                for key, value in data_dictionary['Parameters'].items():
                    pattern = re.compile(r'[.]*Device:[ ]*(?P<device_name>[\W\w]+)[,]+[.]*')
                    matcher = re.match(pattern, value)
                    device_name = matcher.group('device_name')
                    if data_dictionary['Infrastructure'][key] == machine and \
                       data_dictionary['Inference Framework'][key] == inference_framework and \
                       device_name == device and \
                       data_dictionary['Precision'][key] == precision and \
                       (data_dictionary['Mode'][key] in device_precision_modes) == False:
                        device_precision_modes.append(data_dictionary['Mode'][key])
                #print(device_precision_modes)
                framework_device_modes.append(device_precision_modes)
            framework_modes.append(framework_device_modes)
        machine_modes.append(framework_modes)
    execution_modes.append(machine_modes)

print(execution_modes)

[[[[['Sync', 'Async']], [['Sync', 'Async'], ['Sync', 'Async']], [['Sync', 'Async']]]], [[[['Sync', 'Async']], [['Sync', 'Async'], ['Sync', 'Async']], [['Sync', 'Async']]]], [[[['Sync', 'Async']]]]]


In [144]:
rel_row_idx = 4
rel_col_idx = 5

for idx in range(len(infrastructure)):
    row_idx = rel_row_idx
    col_idx = rel_col_idx
    num_cols = 0
    machine = infrastructure[idx]
    # print(machine)
    machine_inference_frameworks = inference_frameworks[idx]
    col_idx3 = col_idx
    for idx2 in range(len(machine_inference_frameworks)):
        machine_framework = machine_inference_frameworks[idx2]
        # print(machine_framework)
        machine_framework_devices = devices[idx][idx2]
        col_idx2 = col_idx
        num_cols2 = 0
        for idx3 in range(len(machine_framework_devices)):
            machine_framework_device = machine_framework_devices[idx3]
            # print(machine_framework_device)
            framework_device_precisions = precisions[idx][idx2][idx3]
            col_idx1 = col_idx
            num_cols1 = 0
            for idx4 in range(len(framework_device_precisions)):
                framework_device_precision = framework_device_precisions[idx4]
                # print(framework_device_precision)
                framework_device_precision_modes = execution_modes[idx][idx2][idx3][idx4]
                for idx5 in range(len(framework_device_precision_modes)):
                    framework_device_precision_mode = framework_device_precision_modes[idx5]
                    # print(framework_device_precision_mode)
                    sheet.write(row_idx, col_idx + idx5, framework_device_precision_mode, cell_format)
                sheet.merge_range(row_idx - 1, col_idx1, \
                                  row_idx - 1, col_idx1 + len(framework_device_precision_modes) - 1, \
                                  framework_device_precision, cell_format)
                col_idx += len(framework_device_precision_modes)
                col_idx1 += len(framework_device_precision_modes)
                num_cols1 += len(framework_device_precision_modes)
                num_cols2 += len(framework_device_precision_modes)
                num_cols += len(framework_device_precision_modes)
            sheet.merge_range(row_idx - 2, col_idx2, \
                              row_idx - 2, col_idx2 + num_cols1 - 1, \
                              machine_framework_device, cell_format)
            col_idx2 += num_cols1
        sheet.merge_range(row_idx - 3, col_idx3, \
                          row_idx - 3, col_idx3 + num_cols2 - 1, \
                          machine_framework, cell_format)
        col_idx3 += num_cols2
            
    sheet.merge_range(row_idx - 4, rel_col_idx, \
                      row_idx - 4, rel_col_idx + num_cols - 1, \
                      machine, cell_format)
    rel_col_idx += num_cols

book.close()